# Imports

In [50]:
import os, sys, time, json, csv, math, difflib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile as tif
from scipy.ndimage.interpolation import zoom

sys.path.append("../../src")
from utils.atlases import reformat, replace_with_common_name, check_for_syn, get_parent_id_from_praid, get_hierarchy, get_dissimilarity

import warnings
warnings.simplefilter('ignore')

# User Inputs

In [51]:
# load label csvs 
labels_dir = '/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/labels/'

# this is the file we want to relate to the PRA
lab_pma = pd.read_csv(os.path.join(labels_dir,'PMA_anns.csv'))

# load PRA files
syns = pd.read_csv(os.path.join(labels_dir,'syns_with_parents.csv'))
PRAlabs = pd.read_csv(os.path.join(labels_dir,'PRA_labels.csv'))

# load json
with open(os.path.join(labels_dir,'PRA_ontology.json')) as f:
 ontology = json.load(f)

# making the PRA atlas label file 

In [103]:
pralab = pd.read_csv('/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/labels/PRA_labels.csv')
syns = pd.read_csv('/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/labels/syns_with_parents.csv')
print(pralab.head())
print(syns.head())

    id                        name  parent_id
0  273            abducens_nucleus        297
1  566  supraoptic_group_accessory        225
2  475           accumbens_nucleus        471
3   78             acoustic_striae         37
4  232    agranular_insular_cortex        232
         common_name               name                   parent
0   abducens_nucleus   abducens_nucleus              medulla_mot
1  accumbens_nucleus     accumbens_core  striatum_ventral_region
2  accumbens_nucleus  accumbens_nucleus  striatum_ventral_region
3  accumbens_nucleus    accumbens_shell  striatum_ventral_region
4  accumbens_nucleus     accumbensshell  striatum_ventral_region


In [104]:
pralab.insert(3,'parent_name','')

In [105]:
x=syns.common_name
manual_check=[]

for idx in pralab.index:
    if pralab['name'][idx] in x.tolist():
        df=syns[syns['common_name']==pralab['name'][idx]]
        pralab['parent_name'][idx]=df['parent'].tolist()[0]
    else:
        manual_check.append(idx)

In [106]:
manual_check

[]

In [107]:
pralab[pralab.index==521]

,id,name,parent_id,parent_name
521,489,superior_colliculus_intermediate_gray,594,superior_colliculus


In [108]:
for idx in pralab.index:
    df = pralab[pralab.name==pralab.parent_name[idx]]
    print('for idx {}, {} was {}'.format(idx, pralab.parent_name[idx],pralab.parent_id[idx]))
    pralab.parent_id[idx]=df['id'].tolist()[0]
    print('      now {}'.format(pralab.parent_id[idx]))

for idx 0, medulla_mot was 297
      now 285
for idx 1, hypothalamus was 225
      now 11
for idx 2, striatum_ventral_region was 471
      now 607
for idx 3, cochlear_nerve was 37
      now 549
for idx 4, insular_cortex was 232
      now 150
for idx 5, agranular_insular_cortex was 157
      now 232
for idx 6, agranular_insular_cortex was 157
      now 232
for idx 7, agranular_insular_cortex was 157
      now 232
for idx 8, fornix_system was 440
      now 579
for idx 9, medulla_mot was 297
      now 285
for idx 10, hippocampal_formation was 579
      now 104
for idx 11, cerebrum was 66
      now 37
for idx 12, cortical_amygdalar_area was 36
      now 575
for idx 13, cortical_amygdalar_area_posterior_part was 575
      now 576
for idx 14, olfactory_areas was 104
      now 105
for idx 15, cerebrum was 37
      now 37
for idx 16, cortical_amygdalar_area was 111
      now 575
for idx 17, amygdalar_nuclei was 605
      now 567
for idx 18, amygdalar_nuclei was 605
      now 567
for idx 19, st

for idx 195, grooves was 37
      now 4
for idx 196, epithalamus was 578
      now 505
for idx 197, epithalamus was 578
      now 505
for idx 198, epithalamus_related was 225
      now 578
for idx 199, epithalamus was 578
      now 505
for idx 200, cerebellar_cortex was 45
      now 51
for idx 201, brainstem was 8
      now 35
for idx 202, brainstem was 577
      now 35
for idx 203, fornix_system was 579
      now 579
for idx 204, hippocampal_commissures was 579
      now 582
for idx 205, hippocampal_formation was 104
      now 104
for idx 206, grooves_of_the_cerebral_cortex was 573
      now 581
for idx 207, cortex was 59
      now 59
for idx 208, hippocampal_formation was 104
      now 104
for idx 209, cranial_nerves was 157
      now 157
for idx 210, medulla_mot was 590
      now 285
for idx 211, hypothalamus was 225
      now 11
for idx 212, hypothalamus was 225
      now 11
for idx 213, cerebrum was 37
      now 37
for idx 214, interbrain was 8
      now 40
for idx 215, interbrain

for idx 387, medulla_mot was 590
      now 285
for idx 388, superior_olive was 601
      now 427
for idx 389, thalamus was 226
      now 10
for idx 390, cortex was 59
      now 59
for idx 391, hypothalamus was 225
      now 11
for idx 392, epithalamus was 505
      now 505
for idx 393, olfactory_areas was 105
      now 105
for idx 394, olfactory_areas was 105
      now 105
for idx 395, cerebrum was 37
      now 37
for idx 396, hindbrain was 190
      now 13
for idx 397, pons was 602
      now 190
for idx 398, pons was 190
      now 190
for idx 399, pons was 601
      now 190
for idx 400, pons was 190
      now 190
for idx 401, pons was 608
      now 190
for idx 402, pons was 190
      now 190
for idx 403, pons_mot was 601
      now 417
for idx 404, pons_mot was 601
      now 417
for idx 405, pons_beh was 598
      now 406
for idx 406, pons_beh was 600
      now 406
for idx 407, grooves_of_the_cerebellar_cortex was 580
      now 580
for idx 408, fiber_tracts was 2
      now 2
for idx 40

for idx 585, thalamus was 226
      now 10
for idx 586, thalamus was 226
      now 10
for idx 587, thalamus was 226
      now 10
for idx 588, thalamus was 226
      now 10
for idx 589, thalamus was 226
      now 10
for idx 590, thalamus was 226
      now 10
for idx 591, thalamus was 226
      now 10
for idx 592, thalamus was 226
      now 10
for idx 593, thalamus was 226
      now 10
for idx 594, thalamus was 226
      now 10
for idx 595, thalamus was 226
      now 10
for idx 596, thalamus was 226
      now 10
for idx 597, thalamus was 226
      now 10
for idx 598, thalamus was 226
      now 10
for idx 599, thalamus was 226
      now 10
for idx 600, thalamus was 226
      now 10
for idx 601, thalamus was 226
      now 10
for idx 602, thalamus was 226
      now 10
for idx 603, thalamus was 226
      now 10
for idx 604, thalamus was 226
      now 10
for idx 605, ventricular_systems was 3
      now 3
for idx 606, pons was 602
      now 190
for idx 607, medulla_sen was 591
      now 267
fo

In [111]:
pralab.to_csv('/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/labels/PRA_labels.csv')

# working on structure id path and parent ids

In [86]:
PRAlabs.insert(4,'family_tree','0')
PRAlabs.insert(5,'family_struct','0')

ValueError: cannot insert family_tree, already exists

In [89]:
PRAlabs

,id,name,parent_id,parent_name,family_tree,family_struct
0,273,abducens_nucleus,285,medulla_mot,"[0, 37, 59, 91, 13, 189, 285, 273]",0
1,566,supraoptic_group_accessory,11,hypothalamus,"[0, 37, 59, 91, 11, 566]",0
2,475,accumbens_nucleus,607,striatum_ventral_region,"[0, 37, 66, 34, 61, 607, 475]",0
3,78,acoustic_striae,549,cochlear_nerve,"[0, 6, 616, 549, 78]",0
4,232,agranular_insular_cortex,150,insular_cortex,"[0, 37, 59, 150, 232]",0
...,...,...,...,...,...,...
622,297,vestibular_nuclei,285,medulla_mot,"[0, 37, 59, 91, 13, 189, 285, 297]",0
623,298,vestibular_nucleus_efferent,285,medulla_mot,"[0, 37, 59, 91, 13, 189, 285, 298]",0
624,616,vestibulocochlear_nerve,6,cochlea,"[0, 6, 616]",0
625,237,visual_cortex,59,cortex,"[0, 37, 59, 237]",0


In [90]:
def get_parent(df,idval):
    parent_id = df[df.id==idval]['parent_id'].tolist()[0]
    return parent_id

def make_struct_from_tree(family_tree):
    family_struct=''
    for i in family_tree:
        family_struct=family_struct+'/{}'.format(i)
    return family_struct

def make_alt_from_tree(family_tree):
    family_struct=''
    for i in family_tree:
        family_struct=family_struct+'.{}'.format(i)
    return family_struct

In [91]:
x=[571,37,0]
x.reverse()
make_struct_from_tree(x)

'/0/37/571'

In [92]:
for idval in PRAlabs.id:
    print(idval)
    idx = PRAlabs[PRAlabs.id==idval].index[0]
    family_tree=[idval]
    parent_id = PRAlabs.parent_id[idx]
    family_tree.append(parent_id)
    while parent_id != 0:
        parent_id = get_parent(PRAlabs,parent_id)
        family_tree.append(parent_id)
    family_tree.reverse()
    family_struct = make_struct_from_tree(family_tree)
    PRAlabs.family_tree[idx]=str(family_tree)
    PRAlabs.family_struct[idx]=family_struct

273
566
475
78
232
618
4
619
160
274
192
63
99
103
344
567
100
106
38
478
345
101
102
479
107
480
108
470
346
109
347
568
171
68
70
71
299
233
511
32
33
413
414
415
66
34
1
368
481
369
79
360
35
217
620
7
8
474
530
416
56
51
9
10
156
184
536
52
46
57
485
58
48
49
537
43
255
127
128
538
539
540
129
541
542
543
544
185
36
569
531
67
570
112
112
37
571
136
253
91
11
12
348
349
76
77
13
72
1
110
452
6
549
300
572
85
86
84
573
574
87
88
89
243
243
59
575
576
244
244
157
545
132
301
134
311
558
559
380
193
130
131
53
563
564
499
80
74
577
116
234
312
275
111
137
138
181
92
139
140
142
141
505
578
90
158
276
117
153
277
342
148
194
2
560
161
172
173
617
159
216
430
162
579
532
93
278
370
118
133
135
55
4
4
580
581
143
144
149
145
44
13
39
163
191
582
182
104
583
119
279
584
585
65
11
225
205
20
21
22
400
388
197
212
389
213
198
377
390
401
206
207
208
196
201
391
586
392
215
199
256
520
209
200
402
615
195
587
331
219
332
222
81
223
220
221
272
280
281
150
235
236
224
40
384
326
167
282
113
1

In [93]:
PRAlabs

,id,name,parent_id,parent_name,family_tree,family_struct
0,273,abducens_nucleus,285,medulla_mot,"[0, 37, 59, 91, 13, 189, 285, 273]",/0/37/59/91/13/189/285/273
1,566,supraoptic_group_accessory,11,hypothalamus,"[0, 37, 59, 91, 11, 566]",/0/37/59/91/11/566
2,475,accumbens_nucleus,607,striatum_ventral_region,"[0, 37, 66, 34, 61, 607, 475]",/0/37/66/34/61/607/475
3,78,acoustic_striae,549,cochlear_nerve,"[0, 6, 616, 549, 78]",/0/6/616/549/78
4,232,agranular_insular_cortex,150,insular_cortex,"[0, 37, 59, 150, 232]",/0/37/59/150/232
...,...,...,...,...,...,...
622,297,vestibular_nuclei,285,medulla_mot,"[0, 37, 59, 91, 13, 189, 285, 297]",/0/37/59/91/13/189/285/297
623,298,vestibular_nucleus_efferent,285,medulla_mot,"[0, 37, 59, 91, 13, 189, 285, 298]",/0/37/59/91/13/189/285/298
624,616,vestibulocochlear_nerve,6,cochlea,"[0, 6, 616]",/0/6/616
625,237,visual_cortex,59,cortex,"[0, 37, 59, 237]",/0/37/59/237


In [94]:
PRAlabs.to_csv('/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/labels/PRA_labels.csv')